# First cleaning our medicalknowlege base

In [1]:
# import os
# import json
# import re

# # Define paths
# base_path = os.getcwd()
# kb_folder_path = os.path.join(base_path, "medicalKnowledgeBase", "Diagnosis_flowchart")
# patient_cases_folder = os.path.join(base_path, "Finished")
# output_file = os.path.join(base_path, "combined_rag_data.json")

# print(f"Knowledge Base Path: {kb_folder_path}")
# print(f"Patient Cases Path: {patient_cases_folder}")

# def clean_text(text):
#     if not isinstance(text, str):
#         return text
#     cleaned = re.sub(r'\s*___\s*', ' ', text)

#     cleaned = re.sub(r' +', ' ', cleaned
#     cleaned = re.sub(r' +([,.:;])', r'\1', cleaned)
#     return cleaned.strip()

# # Counter for generating unique IDs
# kb_id_counter = 0

# def Convertorfunction(data, parent_key=""):
#     global kb_id_counter
#     items = []
#     for key, value in data.items():
#         new_key = f"{parent_key}/{key}" if parent_key else key

#         if isinstance(value, str):
#             kb_id_counter += 1
#             items.append({
#                 "id": f"KB_{kb_id_counter:04d}",
#                 "medicalKB": f"{new_key}: {value}"
#             })
#         elif isinstance(value, list):
#             kb_id_counter += 1
#             items.append({
#                 "id": f"KB_{kb_id_counter:04d}",
#                 "medicalKB": f"{new_key}: []"
#             })
#         elif isinstance(value, dict):
#             items.extend(Convertorfunction(value, new_key))
#     return items

# knowledge_base_entries = []

# if os.path.exists(kb_folder_path):
#     for file_name in os.listdir(kb_folder_path):
#         if file_name.endswith(".json"):
#             file_path = os.path.join(kb_folder_path, file_name)
#             with open(file_path, "r", encoding="utf-8") as f:
#                 data = json.load(f)
#             knowledge_base_entries.extend(Convertorfunction(data))
#     print(f"Processed {len(knowledge_base_entries)} knowledge base entries")
# else:
#     print(f"Warning: Knowledge base folder not found at {kb_folder_path}")

# def flatten_reasoning(obj, prefix=""):
#     parts = []
#     if isinstance(obj, dict):
#         for key, value in obj.items():
#             new_prefix = f"{prefix} -> {key}" if prefix else key
#             if isinstance(value, dict) and value:  # Non-empty dict
#                 parts.append(flatten_reasoning(value, new_prefix))
#             else:
#                 parts.append(new_prefix)
#     return " | ".join(filter(None, parts))

# def process_patient_case(file_path, file_name, disease_group, specific_disease=None):
#     with open(file_path, 'r', encoding='utf-8') as f:
#         try:
#             data = json.load(f)
#         except json.JSONDecodeError:
#             print(f"Error decoding JSON: {file_path}")
#             return None

#     inputs = {}
#     reasoning = {}
#     for key, value in data.items():
#         if key.lower().startswith("input"):
#             inputs[key] = value
#         else:
#             reasoning[key] = value

#     cleaned_inputs = {}
#     for i in range(1, 7):
#         key = f"input{i}"
#         val = None
#         for k in inputs:
#             if k.lower() == key:
#                 val = inputs[k]
#                 break
        
#         if not val or (isinstance(val, str) and val.strip() == ""):
#             val = "NA"
        
#         if isinstance(val, str):
#             cleaned_inputs[key] = clean_text(val.strip())
#         else:
#             cleaned_inputs[key] = val

#     reasoning_string = flatten_reasoning(reasoning)
#     # here i am using file name asid 
#     case_id = file_name.replace(".json", "")
    
#     case_entry = {
#         "id": case_id,
#         "patient_case": {
#             "disease_group": disease_group,
#             "specific_disease": specific_disease if specific_disease else "NA",
#             "reasoning": reasoning_string,
#             "inputs": cleaned_inputs
#         }
#     }
    
#     return case_entry

# patient_cases_entries = []

# if os.path.exists(patient_cases_folder):
#     for root, dirs, files in os.walk(patient_cases_folder):
#         for file in files:
#             if file.endswith(".json"):
#                 file_path = os.path.join(root, file)
                
#                 rel_path = os.path.relpath(root, patient_cases_folder)
#                 path_parts = rel_path.split(os.sep)
                
#                 # Get disease group and specific disease from folder structure
#                 if len(path_parts) >= 1 and path_parts[0] != ".":
#                     disease_group = path_parts[0]
#                     specific_disease = path_parts[1] if len(path_parts) > 1 else None
                    
#                     case_data = process_patient_case(file_path, file, disease_group, specific_disease)
#                     if case_data:
#                         patient_cases_entries.append(case_data)
#     print(f"Processed {len(patient_cases_entries)} patient cases")
# else:
#     print(f"Warning: Patient cases folder not found at {patient_cases_folder}")

# # Combine knowledge base entries (upper part) with patient cases (lower part)
# combined_data = knowledge_base_entries + patient_cases_entries

# # Save the combined output
# with open(output_file, "w", encoding="utf-8") as f:
#     json.dump(combined_data, f, indent=2, ensure_ascii=False)

# print(f"\n=== Summary ===")
# print(f"Total Knowledge Base entries: {len(knowledge_base_entries)} (IDs: KB_0001 to KB_{kb_id_counter:04d})")
# print(f"Total Patient Cases: {len(patient_cases_entries)}")
# print(f"Combined entries: {len(combined_data)}")
# print(f"Saved to: {output_file}")

# Setting up Env

In [2]:
import json
import os
import numpy as np
from typing import List, Dict
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer, util


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\Muhammad Abu Huraira\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


C:\Users\Muhammad Abu Huraira\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
!pip install transformers accelerate huggingface-hub sentence-transformers faiss-cpu streamlit python-dotenv --quiet



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\Muhammad Abu Huraira\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
# using this lightweight sentence transformer
MODEL_NAME = 'all-MiniLM-L6-v2'
TOP_K=4

# RAG Pipeline

In [5]:
def load_data(file_name):
    with open(file_name,"r",encoding='utf-8') as f:
        data=json.load(f)
    return data

def generate_vector_embedding(model, kb_items):
    print("Creating Vector embeddings of the json file")
    texts = [f"{item.get('id','')} : {item.get('medicalKB','')}" for item in kb_items]
    embeddings = model.encode(texts, convert_to_tensor=True)
    return embeddings

def retrieve_from_kb(model, query_text, kb_embeddings, kb_items, k=TOP_K):
    query_emb = model.encode(query_text, convert_to_tensor=True)

    cos_scores = util.cos_sim(query_emb, kb_embeddings)[0]
    scores = cos_scores.cpu().numpy()

    k = min(k, len(scores))
    top_idx = np.argpartition(-scores, range(k))[:k]
    top_idx = top_idx[np.argsort(-scores[top_idx])]

    results = []
    for idx in top_idx:
        results.append((
            kb_items[int(idx)],
            float(scores[int(idx)])
        ))

    return results

def construct_prompt(patient_case, retrieved_kb):
    context_str = "Medical Knowledge:\n"
    for item, score in retrieved_kb:
        context_str += f"- [{item.get('id','')}] {item.get('medicalKB','')} (Relevance: {score:.2f})\n"

    case_str = f"Patient Case Details:\n{patient_case['input_text']}"

    prompt = f"""
You are an expert medical diagnostician.
Use the provided medical knowledge to explain the patient case.

{context_str}

{case_str}

Give reasoning and final answer.
"""
    return prompt



# Preparing HuggingFace LLM

In [6]:
from transformers import AutoTokenizer,AutoModelForCausalLM
import torch
LLM_Model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
tokenizer=AutoTokenizer.from_pretrained(LLM_Model,use_fast=False)
llm=AutoModelForCausalLM.from_pretrained(LLM_Model,torch_dtype=torch.float16,device_map="auto")

Loading checkpoint shards: 100%|██████████| 2/2 [00:56<00:00, 28.43s/it]
Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [7]:
def llm_generate(prompt, max_new_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt").to(llm.device)

    output = llm.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [8]:
def run_rag(patient_case, kb_items, kb_embeddings):
    query = patient_case["input_text"]

    retrieved = retrieve_from_kb(
        model,
        query,
        kb_embeddings,
        kb_items,
        TOP_K
    )

    prompt = construct_prompt(patient_case, retrieved)
    answer = llm_generate(prompt)

    return answer, retrieved, prompt


# Model Inference

In [9]:
kb_items = load_data("mimic-iv-ext-direct-1.0.0\My_dataset\combined_rag_data.json")
model = SentenceTransformer(MODEL_NAME)

kb_embeddings = generate_vector_embedding(model, kb_items)

patient_case = {
    "input_text": "A 60-year-old male with sudden chest pain radiating to back..."
}

answer, retrieved, prompt = run_rag(patient_case, kb_items, kb_embeddings)

print(answer)


<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
C:\Users\Muhammad Abu Huraira\AppData\Local\Temp\ipykernel_8740\350318453.py:1: SyntaxWarning: invalid escape sequence '\M'
  kb_items = load_data("mimic-iv-ext-direct-1.0.0\My_dataset\combined_rag_data.json")
C:\Users\Muhammad Abu Huraira\AppData\Local\Temp\ipykernel_8740\350318453.py:1: SyntaxWarning: invalid escape sequence '\M'
  kb_items = load_data("mimic-iv-ext-direct-1.0.0\My_dataset\combined_rag_data.json")


FileNotFoundError: [Errno 2] No such file or directory: 'mimic-iv-ext-direct-1.0.0\\My_dataset\\combined_rag_data.json'

# Using API Models

In [ ]:
!pip install google-generativeai python-dotenv --quiet

import os
from dotenv import load_dotenv
import google.generativeai as genai

# Get API key
api_key = "APIKey"


genai.configure(api_key=api_key)

# Example prompt
prompt = """
You are a medical assistant.
First explain what it diesase could be then give all the reasons
Explain the symptoms of a 60-year-old male with sudden chest pain radiating to the back.
"""

# Generate response
model = genai.GenerativeModel("gemini-2.0-flash")
response = model.generate_content(prompt)

print(response.text)

